In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib

# Carregar o CSV com nome atualizado
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')

# Converter o timestamp para datetime
if 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'], errors='coerce')  # Remove unit='s' e converte diretamente
    # Verifique se houve falha na conversão
    if df['time'].isnull().any():
        num_failed = df['time'].isnull().sum()
        print(f"Atenção: {num_failed} entradas na coluna 'time' não puderam ser convertidas para datetime.")

# Atribuir os dados às variáveis
timestamp = df['time']
open_price = df['open']
high_price = df['high']
low_price = df['low']
close_price = df['close']

# Parâmetros configuráveis
emaShortLength = 11  # Período da EMA Curta
emaLongLength = 55  # Período da EMA Longa
rsiLength = 22  # Período do RSI
macdShort = 15  # Período Curto MACD
macdLong = 34  # Período Longo MACD
macdSignal = 11  # Período de Sinal MACD
adxLength = 16  # Período ADX (igual ao DI Length)
adxSmoothing = 13  # ADX Smoothing
adxThreshold = 12  # Nível de ADX para indicar tendência
bbLength = 14  # Período do Bollinger Bands
bbMultiplier = 1.7  # Multiplicador do Bollinger Bands
lateralThreshold = 0.005  # Limite de Lateralização
stopgain_lateral_long = 1.11
stoploss_lateral_long = 0.973
stopgain_lateral_short = 0.973
stoploss_lateral_short = 1.09
stopgain_normal_long = 1.32
stoploss_normal_long = 0.92
stopgain_normal_short = 0.77
stoploss_normal_short = 1.12

# Funções para cálculo manual do MACD
def macd_func(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    tr = []
    previous_close = close.iloc[0]
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        current_plus_dm = high.iloc[i] - high.iloc[i-1]
        current_minus_dm = low.iloc[i-1] - low.iloc[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)

        tr1 = high.iloc[i] - low.iloc[i]
        tr2 = abs(high.iloc[i] - previous_close)
        tr3 = abs(low.iloc[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)

        previous_close = close.iloc[i]

    tr.insert(0, np.nan)
    plus_dm.insert(0, np.nan)
    minus_dm.insert(0, np.nan)

    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)

    atr = tr.rolling(window=di_lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.ewm(alpha=1/adx_smoothing, adjust=False).mean()

    return plus_di, minus_di, adx

# Cálculo dos indicadores
emaShort = talib.EMA(close_price, emaShortLength)
emaLong = talib.EMA(close_price, emaLongLength)
rsi = talib.RSI(close_price, timeperiod=rsiLength)
plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
adx = adx.fillna(0).astype(int)
macdLine, signalLine, macdHist = macd_func(close_price, macdShort, macdLong, macdSignal)
upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)

# Função para detectar crossover (cruzamento ascendente)
def crossover(series1, series2):
    return (series1.shift(1) <= series2.shift(1)) & (series1 > series2)

# Função para detectar crossunder (cruzamento descendente)
def crossunder(series1, series2):
    return (series1.shift(1) >= series2.shift(1)) & (series1 < series2)

# Condição de mercado em tendência (baseada no ADX)
trendingMarket = adx >= adxThreshold

# Condições de lateralização com Bollinger Bands
bandWidth = (upperBand - lowerBand) / middleBand
isLateral = bandWidth < lateralThreshold

# Saldo inicial
saldo = 1_000_000  # Saldo inicial de 1.000.000
quantidade = 0  # Quantidade de BTC comprada/vendida na transação
position_open = False  # Indica se estamos em uma transação
current_position = None  # 'long' ou 'short'
entry_price = None  # Preço de entrada da posição
orders = []
trade_count = 0  # Contador de trades

# Condição Long
longCondition = (crossover(emaShort, emaLong)) & (rsi < 60) & (macdHist > 0.5) & trendingMarket

# Condição Short
shortCondition = (crossunder(emaShort, emaLong)) & (rsi > 40) & (macdHist < -0.5) & trendingMarket

commission = 0.00032  # 0,032%

# Definir período de negociação
start_date = pd.to_datetime('2024-02-01 00:00:00')  # Data inicial
end_date = pd.to_datetime('2024-09-01 23:59:59')    # Data final

# Loop para verificar e executar as ordens
for i in range(1, len(df)):
    current_time = timestamp.iloc[i]

    # Verificar se a data atual está dentro do período de negociação
    if current_time < start_date:
        continue  # Pula para a próxima iteração se antes do período de negociação
    if current_time > end_date:
        # Fechar qualquer posição aberta no final do período
        if position_open:
            # Fechar posição na abertura deste candle
            exit_price = open_price.iloc[i]
            if current_position == 'long':
                saldo = quantidade * exit_price
                orders.append(f"Sair de transação (long) em {current_time} com preço {exit_price:.2f} (Fechamento do período), saldo atualizado: {saldo:.2f}")
            elif current_position == 'short':
                saldo += quantidade * (entry_price - exit_price)
                orders.append(f"Sair de transação (short) em {current_time} com preço {exit_price:.2f} (Fechamento do período), saldo atualizado: {saldo:.2f}")
            position_open = False
        break  # Sai do loop se passamos do período de negociação

    adjusted_timestamp = timestamp.iloc[i]

    print(f"Vela: {adjusted_timestamp}")
    print(f"EMA Curta: {emaShort[i]}, EMA Longa: {emaLong[i]}")
    print(f"RSI: {rsi[i]}")
    print(f"MACD Line: {macdLine[i]}, Signal Line: {signalLine[i]}, MACD Hist: {macdHist[i]}")
    print(f"ADX: {adx[i]}")
    print(f"Bollinger Upper: {upperBand[i]}, Bollinger Lower: {lowerBand[i]}")
    print("-" * 50)
    
    # Determinar a condição de lateralização da vela anterior
    lateral_prev = isLateral.iloc[i-1]

    # Estratégia de Mean Reversion para mercado lateral
    if lateral_prev:
        if not position_open:
            # Condição para entrar em Long
            if (close_price.iloc[i-1] < lowerBand.iloc[i-1]) and crossover(close_price, lowerBand).iloc[i-1] and longCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]  # Executa no candle atual
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (long lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                position_open = True
                current_position = 'long'
                trade_count += 1
                print(f"Trade LONG lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            # Condição para entrar em Short
            elif (close_price.iloc[i-1] > upperBand.iloc[i-1]) and crossunder(close_price, upperBand).iloc[i-1] and shortCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (short lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                position_open = True
                current_position = 'short'
                trade_count += 1
                print(f"Trade SHORT lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
        elif position_open:
            if current_position == 'long':
                # Condição para inverter para Short
                if (close_price.iloc[i] > upperBand.iloc[i]) and crossover(close_price, upperBand).iloc[i] and shortCondition.iloc[i-1]:
                    saldo = quantidade * close_price.iloc[i]
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Short), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]  # Executa no candle atual
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif current_position == 'short':
                # Condição para inverter para Long
                if (close_price.iloc[i] < lowerBand.iloc[i]) and crossover(close_price, lowerBand).iloc[i] and longCondition.iloc[i-1]:
                    saldo += quantidade * (entry_price - close_price.iloc[i])
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Long), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")

            # Gerenciamento de posições abertas com base nos parâmetros laterais da vela anterior
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                if low_price.iloc[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss LONG atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                elif high_price.iloc[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit LONG atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short  # Corrigido para takeProfitShort
                if high_price.iloc[i] >= stopLossShort:
                    saldo -= quantidade * (stopLossShort - entry_price)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {stopLossShort:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss SHORT atingido em {adjusted_timestamp} a {stopLossShort:.2f}")
                elif low_price.iloc[i] <= takeProfitShort:
                    saldo += quantidade * (entry_price - takeProfitShort)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit SHORT atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")

    # Estratégia para mercado não lateral
    if not lateral_prev:
        if not position_open:
            # Condição para entrar em Long
            if longCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (long normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                position_open = True
                current_position = 'long'
                trade_count += 1
                print(f"Trade LONG normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            # Condição para entrar em Short
            elif shortCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (short normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                position_open = True
                current_position = 'short'
                trade_count += 1
                print(f"Trade SHORT normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
        elif position_open:
            if current_position == 'long':
                # Condição para inverter para Short
                if shortCondition.iloc[i-1]:
                    saldo = quantidade * open_price.iloc[i]
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Short), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif current_position == 'short':
                # Condição para inverter para Long
                if longCondition.iloc[i-1]:
                    saldo += quantidade * (entry_price - open_price.iloc[i])
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Long), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")

            # Gerenciamento de posições abertas com base nos parâmetros não laterais
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                if low_price.iloc[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss LONG normal atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                elif high_price.iloc[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit LONG normal atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short  # Corrigido para takeProfitShort
                if high_price.iloc[i] >= stopLossShort:
                    saldo -= quantidade * (stopLossShort - entry_price)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss SHORT normal atingido em {adjusted_timestamp} a {stopLossShort:.2f}")
                elif low_price.iloc[i] <= takeProfitShort:
                    saldo += quantidade * (entry_price - takeProfitShort)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit SHORT normal atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")

# Exibir as ordens geradas
for order in orders:
    print(order)

# Exibir o número total de trades
print(f"Total de trades: {trade_count}")


Vela: 2024-02-01 00:00:00
EMA Curta: 42786.82392074956, EMA Longa: 42890.67607083344
RSI: 42.91480817623315
MACD Line: -95.08829080405121, Signal Line: -31.11292244716027, MACD Hist: -63.97536835689094
ADX: 22
Bollinger Upper: 43635.91743805733, Bollinger Lower: 42183.33970479966
--------------------------------------------------
Vela: 2024-02-01 01:00:00
EMA Curta: 42649.03660062463, EMA Longa: 42857.44121116082
RSI: 37.56965292706714
MACD Line: -149.21424935218238, Signal Line: -50.796476931330616, MACD Hist: -98.41777242085176
ADX: 24
Bollinger Upper: 43693.01597501105, Bollinger Lower: 42031.19831070308
--------------------------------------------------
Vela: 2024-02-01 02:00:00
EMA Curta: 42565.197167187194, EMA Longa: 42832.0325964765
RSI: 40.54333627888175
MACD Line: -180.18770623044838, Signal Line: -72.36168181451691, MACD Hist: -107.82602441593147
ADX: 25
Bollinger Upper: 43712.288207832185, Bollinger Lower: 41946.854649310524
-------------------------------------------------

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib

# Carregar o CSV com nome atualizado
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')

# Converter o timestamp para datetime (descomente se necessário)
if 'time' in df.columns:
    try:
        df['time'] = pd.to_datetime(df['time'], unit='s')
    except:
        pass  # Se o timestamp já estiver em formato datetime

# Atribuir os dados às variáveis
timestamp = df['time']
open_price = df['open']
high_price = df['high']
low_price = df['low']
close_price = df['close']

# Parâmetros configuráveis
emaShortLength = 11  # Período da EMA Curta
emaLongLength = 55  # Período da EMA Longa
rsiLength = 22  # Período do RSI
macdShort = 15  # Período Curto MACD
macdLong = 34  # Período Longo MACD
macdSignal = 11  # Período de Sinal MACD
adxLength = 16  # Período ADX (igual ao DI Length)
adxSmoothing = 13  # ADX Smoothing
adxThreshold = 12  # Nível de ADX para indicar tendência
bbLength = 14  # Período do Bollinger Bands
bbMultiplier = 1.7  # Multiplicador do Bollinger Bands
lateralThreshold = 0.005  # Limite de Lateralização
stopgain_lateral_long = 1.11
stoploss_lateral_long = 0.973
stopgain_lateral_short = 0.973
stoploss_lateral_short = 1.09
stopgain_normal_long = 1.32
stoploss_normal_long = 0.92
stopgain_normal_short = 0.77
stoploss_normal_short = 1.12

# Funções para cálculo manual do MACD
def macd_func(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    tr = []
    previous_close = close.iloc[0]
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        current_plus_dm = high.iloc[i] - high.iloc[i-1]
        current_minus_dm = low.iloc[i-1] - low.iloc[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)

        tr1 = high.iloc[i] - low.iloc[i]
        tr2 = abs(high.iloc[i] - previous_close)
        tr3 = abs(low.iloc[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)

        previous_close = close.iloc[i]

    tr.insert(0, np.nan)
    plus_dm.insert(0, np.nan)
    minus_dm.insert(0, np.nan)

    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)

    atr = tr.rolling(window=di_lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.ewm(alpha=1/adx_smoothing, adjust=False).mean()

    return plus_di, minus_di, adx

# Cálculo dos indicadores
emaShort = talib.EMA(close_price, emaShortLength)
emaLong = talib.EMA(close_price, emaLongLength)
rsi = talib.RSI(close_price, timeperiod=rsiLength)
plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
adx = adx.fillna(0).astype(int)
macdLine, signalLine, macdHist = macd_func(close_price, macdShort, macdLong, macdSignal)
upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)

# Função para detectar crossover (cruzamento ascendente)
def crossover(series1, series2):
    return (series1.shift(1) <= series2.shift(1)) & (series1 > series2)

# Função para detectar crossunder (cruzamento descendente)
def crossunder(series1, series2):
    return (series1.shift(1) >= series2.shift(1)) & (series1 < series2)

# Condição de mercado em tendência (baseada no ADX)
trendingMarket = adx >= adxThreshold

# Condições de lateralização com Bollinger Bands
bandWidth = (upperBand - lowerBand) / middleBand
isLateral = bandWidth < lateralThreshold

# Determinar eventos de cruzamento
crossover_events = crossover(emaShort, emaLong)
crossunder_events = crossunder(emaShort, emaLong)

# Inicializar a Série ema_cross_event
ema_cross_event = pd.Series(index=df.index, dtype='object')

ema_cross_event[crossover_events] = 'crossover'
ema_cross_event[crossunder_events] = 'crossunder'

# Preencher adiante para obter o último evento de cruzamento em cada momento
last_cross_event = ema_cross_event.fillna(method='ffill')

# Detectar mudanças na condição de mercado
isLateral_change = isLateral != isLateral.shift(1)

# Saldo inicial
saldo = 1_000_000  # Saldo inicial de 1.000.000
quantidade = 0  # Quantidade de BTC comprada/vendida na transação
position_open = False  # Indica se estamos em uma transação
current_position = None  # 'long' ou 'short'
entry_price = None  # Preço de entrada da posição
orders = []
trade_count = 0  # Contador de trades

commission = 0.00032  # 0,032%

# Loop para verificar e executar as ordens
for i in range(1, len(df)):
    adjusted_timestamp = timestamp.iloc[i]

    # Determinar a condição da vela anterior e atual
    lateral_prev = isLateral.iloc[i-1]
    lateral_curr = isLateral.iloc[i]

    # Verificar se a condição de mercado mudou
    if isLateral_change.iloc[i]:
        # A condição de mercado mudou
        last_event = last_cross_event.iloc[i-1]
        if not position_open:
            if last_event == 'crossover' and rsi.iloc[i-1] < 60 and macdHist.iloc[i-1] > 0.5:
                # Entrar em posição Long
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                if lateral_curr:
                    # Mercado lateral
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                else:
                    # Mercado em tendência
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (long) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                position_open = True
                current_position = 'long'
                trade_count += 1
                print(f"Trade LONG executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif last_event == 'crossunder' and rsi.iloc[i-1] > 40 and macdHist.iloc[i-1] < -0.5:
                # Entrar em posição Short
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                if lateral_curr:
                    # Mercado lateral
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                else:
                    # Mercado em tendência
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (short) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                position_open = True
                current_position = 'short'
                trade_count += 1
                print(f"Trade SHORT executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
        elif position_open:
            if current_position == 'long':
                if last_event == 'crossunder' and rsi.iloc[i-1] > 40 and macdHist.iloc[i-1] < -0.5:
                    # Reverter para posição Short
                    saldo = quantidade * open_price.iloc[i]
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Short), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    if lateral_curr:
                        stopLossShort = entry_price * stoploss_lateral_short
                        takeProfitShort = entry_price * stopgain_lateral_short
                    else:
                        stopLossShort = entry_price * stoploss_normal_short
                        takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif current_position == 'short':
                if last_event == 'crossover' and rsi.iloc[i-1] < 60 and macdHist.iloc[i-1] > 0.5:
                    # Reverter para posição Long
                    saldo += quantidade * (entry_price - open_price.iloc[i])
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Long), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    if lateral_curr:
                        stopLossLong = entry_price * stoploss_lateral_long
                        takeProfitLong = entry_price * stopgain_lateral_long
                    else:
                        stopLossLong = entry_price * stoploss_normal_long
                        takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
    else:
        # Estratégia padrão baseada na condição atual de mercado
        # Estratégia de Mean Reversion para mercado lateral
        if lateral_prev:
            if not position_open:
                # Condição para entrar em Long
                if (close_price.iloc[i-1] < lowerBand.iloc[i-1]) and crossover(close_price, lowerBand).iloc[i-1]:
                    entry_price = open_price.iloc[i]  # Executa no candle atual
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
                # Condição para entrar em Short
                elif (close_price.iloc[i-1] > upperBand.iloc[i-1]) and crossunder(close_price, upperBand).iloc[i-1]:
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif position_open:
                # Gerenciamento de posições abertas com base nos parâmetros laterais
                if current_position == 'long':
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    if low_price.iloc[i] <= stopLossLong:
                        saldo = quantidade * stopLossLong
                        orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Stop Loss LONG atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                    elif high_price.iloc[i] >= takeProfitLong:
                        saldo = quantidade * takeProfitLong
                        orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Take Profit LONG atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
                elif current_position == 'short':
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    if high_price.iloc[i] >= stopLossShort:
                        saldo -= quantidade * (stopLossShort - entry_price)
                        orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {stopLossShort:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Stop Loss SHORT atingido em {adjusted_timestamp} a {stopLossShort:.2f}")
                    elif low_price.iloc[i] <= takeProfitShort:
                        saldo += quantidade * (entry_price - takeProfitShort)
                        orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Take Profit SHORT atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")
        else:
            # Estratégia para mercado em tendência
            if not position_open:
                # Condição para entrar em Long
                if longCondition.iloc[i-1]:
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
                # Condição para entrar em Short
                elif shortCondition.iloc[i-1]:
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif position_open:
                # Gerenciamento de posições abertas com base nos parâmetros não laterais
                if current_position == 'long':
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    if low_price.iloc[i] <= stopLossLong:
                        saldo = quantidade * stopLossLong
                        orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Stop Loss LONG normal atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                    elif high_price.iloc[i] >= takeProfitLong:
                        saldo = quantidade * takeProfitLong
                        orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Take Profit LONG normal atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
                elif current_position == 'short':
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    if high_price.iloc[i] >= stopLossShort:
                        saldo -= quantidade * (stopLossShort - entry_price)
                        orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {stopLossShort:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Stop Loss SHORT normal atingido em {adjusted_timestamp} a {stopLossShort:.2f}")
                    elif low_price.iloc[i] <= takeProfitShort:
                        saldo += quantidade * (entry_price - takeProfitShort)
                        orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                        position_open = False
                        print(f"Take Profit SHORT normal atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")

# Exibir as ordens geradas
for order in orders:
    print(order)

# Exibir o número total de trades
print(f"Total de trades: {trade_count}")


C:\Users\erik\AppData\Local\Temp\ipykernel_13876\4147341913.py:127: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  last_cross_event = ema_cross_event.fillna(method='ffill')


Trade SHORT normal executado em 2020-04-01 07:00:00 a 6289.50
Stop Loss SHORT normal atingido em 2020-04-02 17:00:00 a 7044.24
Trade SHORT normal executado em 2020-04-10 04:00:00 a 7175.50
Stop Loss SHORT normal atingido em 2020-04-29 10:00:00 a 8036.56
Trade SHORT normal executado em 2020-05-04 01:00:00 a 8806.00
Stop Loss SHORT normal atingido em 2020-05-07 17:00:00 a 9862.72
Trade SHORT normal executado em 2020-05-09 23:00:00 a 9604.50
Trade LONG executado em 2020-06-01 14:00:00 a 9550.00
Trade SHORT executado em 2020-06-03 06:00:00 a 9513.00
Trade LONG executado em 2020-06-21 10:00:00 a 9366.50
Stop Loss LONG atingido em 2020-06-27 15:00:00 a 9113.60
Trade LONG normal executado em 2020-06-29 02:00:00 a 9144.00
Trade SHORT executado em 2020-07-11 22:00:00 a 9217.00
Trade LONG executado em 2020-07-12 12:00:00 a 9240.00
Trade SHORT executado em 2020-07-15 18:00:00 a 9205.50
Trade LONG executado em 2020-07-20 06:00:00 a 9185.50
Take Profit LONG normal atingido em 2020-08-17 13:00:00 a 

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib
from itertools import product
from tqdm import tqdm  # Para mostrar uma barra de progresso
import random

# Função para calcular o MACD manualmente
def macd(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

# Função para calcular o ADX manualmente
def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    tr = []
    previous_close = close[0]  # Primeiro valor de fechamento
    
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        # Calcular +DM e -DM
        current_plus_dm = high[i] - high[i-1]
        current_minus_dm = low[i-1] - low[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)
        
        # Calcular True Range (TR)
        tr1 = high[i] - low[i]
        tr2 = abs(high[i] - previous_close)
        tr3 = abs(low[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)
        
        # Atualizar o fechamento anterior
        previous_close = close[i]
    
    # Adiciona um valor NaN para a primeira posição
    tr.insert(0, None)
    plus_dm.insert(0, None)
    minus_dm.insert(0, None)
    
    # Converter listas para Series do pandas
    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)
    
    # Calcular o ATR usando o lookback fornecido para DI
    atr = tr.rolling(window=di_lookback).mean()
    
    # Calcular DI+ e DI- com o lookback de DI
    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    
    # Calcular o DX
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    
    # Calcular o ADX usando o smoothing específico para o ADX
    adx = dx.ewm(alpha=1/adx_smoothing).mean()
    
    return plus_di, minus_di, adx

# Função para detectar crossover (cruzamento ascendente)
def crossover(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] > series2_vals[1:]) & (series1_vals[:-1] <= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False para alinhar os comprimentos
    return pd.Series(cross, index=series1.index)

# Função para detectar crossunder (cruzamento descendente)
def crossunder(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] < series2_vals[1:]) & (series1_vals[:-1] >= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False para alinhar os comprimentos
    return pd.Series(cross, index=series1.index)

# Função de Backtest
def backtest(params, df):
    # Desempacotar parâmetros
    (emaShortLength, emaLongLength, rsiLength, macdShort, macdLong, macdSignal,
     adxLength, adxSmoothing, adxThreshold, bbLength, bbMultiplier,
     lateralThreshold, stopgain_lateral_long, stoploss_lateral_long,
     stopgain_lateral_short, stoploss_lateral_short,
     stopgain_normal_long, stoploss_normal_long,
     stopgain_normal_short, stoploss_normal_short,
     commission) = params
    
    # Atribuir os dados às variáveis
    timestamp = df['time']
    open_price = df['open']
    high_price = df['high']
    low_price = df['low']
    close_price = df['close']
    
    # Cálculo dos Indicadores
    emaShort = talib.EMA(close_price, emaShortLength).round(0)
    emaLong = talib.EMA(close_price, emaLongLength).round(0)
    rsi = talib.RSI(close_price, timeperiod=rsiLength)
    plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
    adx = adx.fillna(0).astype(int)
    macdLine, signalLine, macdHist = macd(close_price, macdShort, macdLong, macdSignal)
    upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)
    
    # Condição de mercado em tendência (baseada no ADX)
    trendingMarket = adx >= adxThreshold
    
    # Condições de lateralização com Bollinger Bands
    bandWidth = (upperBand - lowerBand) / middleBand
    isLateral = bandWidth < lateralThreshold
    
    # Condições de crossover
    longCondition = (crossover(emaShort, emaLong)) & (rsi < 60) & (macdHist > 0.5) & trendingMarket
    shortCondition = (crossunder(emaShort, emaLong)) & (rsi > 40) & (macdHist < -0.5) & trendingMarket
    
    # Inicialização do Backtest
    saldo = 1_000_000  # Saldo inicial de 1.000.000
    quantidade = 0  # Quantidade de BTC comprada/vendida na transação
    position_open = False  # Indica se estamos em uma transação
    current_position = None  # 'long' ou 'short'
    entry_price = None  # Preço de entrada da posição
    trade_count = 0  # Contador de trades
    
    # Loop para verificar e executar as ordens
    for i in range(len(df)):
        # Estratégia de Mean Reversion para mercado lateral
        if isLateral[i]:
            if not position_open:
                if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i] and longCondition[i]:
                    entry_price = close_price[i]
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na compra
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                elif (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i] and shortCondition[i]:
                    entry_price = close_price[i]
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na venda
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
            elif position_open:
                if current_position == 'long':
                    if (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i] and shortCondition[i]:
                        saldo = quantidade * close_price[i]
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Short
                        entry_price = close_price[i]
                        stopLossShort = entry_price * stoploss_lateral_short
                        takeProfitShort = entry_price * stopgain_lateral_short
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'short'
                        trade_count += 1
                elif current_position == 'short':
                    if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i] and longCondition[i]:
                        saldo = saldo + (quantidade * (entry_price - close_price[i]))
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Long
                        entry_price = close_price[i]
                        stopLossLong = entry_price * stoploss_lateral_long
                        takeProfitLong = entry_price * stopgain_lateral_long
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'long'
                        trade_count += 1

        # Estratégia para mercado não lateral
        if not isLateral[i]:
            if not position_open:
                if longCondition[i]:
                    entry_price = close_price[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na compra
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                elif shortCondition[i]:
                    entry_price = close_price[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na venda
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
            elif position_open:
                if current_position == 'long':
                    if shortCondition[i]:
                        saldo = quantidade * close_price[i]
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Short
                        entry_price = close_price[i]
                        stopLossShort = entry_price * stoploss_normal_short
                        takeProfitShort = entry_price * stopgain_normal_short
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'short'
                        trade_count += 1
                elif current_position == 'short':
                    if longCondition[i]:
                        saldo = saldo + (quantidade * (entry_price - close_price[i]))
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Long
                        entry_price = close_price[i]
                        stopLossLong = entry_price * stoploss_normal_long
                        takeProfitLong = entry_price * stopgain_normal_long
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'long'
                        trade_count += 1

                # Gerenciamento de Stop Loss e Take Profit
                if current_position == 'long':
                    if low_price[i] <= stopLossLong:
                        saldo = quantidade * stopLossLong
                        position_open = False
                        current_position = None
                    elif high_price[i] >= takeProfitLong:
                        saldo = quantidade * takeProfitLong
                        position_open = False
                        current_position = None
                elif current_position == 'short':
                    if high_price[i] >= stopLossShort:
                        saldo = saldo - (quantidade * (stopLossShort - entry_price))
                        position_open = False
                        current_position = None
                    elif low_price[i] <= takeProfitShort:
                        saldo = saldo + (quantidade * (entry_price - takeProfitShort))
                        position_open = False
                        current_position = None

    # Fechar posição aberta no final do backtest
    if position_open:
        if current_position == 'long':
            saldo = quantidade * close_price.iloc[-1]
        elif current_position == 'short':
            saldo = saldo + (quantidade * (entry_price - close_price.iloc[-1]))
    
    return saldo, trade_count

# Função para gerar listas de parâmetros com base em [início, fim, passo]
def generate_parameter_list(start, end, step, decimals=0):
    if isinstance(start, float) or isinstance(end, float) or isinstance(step, float):
        num = int(np.floor((end - start) / step)) + 1
        return [round(start + step * i, decimals) for i in range(num)]
    else:
        return list(range(start, end + 1, step))

# Função para realizar Random Search
def random_search(df, param_grid, n_iterations, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    keys = list(param_grid.keys())
    results = []
    
    print(f"Iniciando Random Search com {n_iterations} iterações...")
    
    for _ in tqdm(range(n_iterations), desc="Random Search"):
        # Selecionar aleatoriamente um valor para cada parâmetro
        params = {key: random.choice(values) for key, values in param_grid.items()}
        
        # Criar uma tupla de parâmetros na ordem esperada pela função backtest
        param_tuple = (
            params['emaShortLength'],
            params['emaLongLength'],
            params['rsiLength'],
            params['macdShort'],
            params['macdLong'],
            params['macdSignal'],
            params['adxLength'],
            params['adxSmoothing'],
            params['adxThreshold'],
            params['bbLength'],
            params['bbMultiplier'],
            params['lateralThreshold'],
            params['stopgain_lateral_long'],
            params['stoploss_lateral_long'],
            params['stopgain_lateral_short'],
            params['stoploss_lateral_short'],
            params['stopgain_normal_long'],
            params['stoploss_normal_long'],
            params['stopgain_normal_short'],
            params['stoploss_normal_short'],
            params['commission']
        )
        
        # Executar o backtest com os parâmetros selecionados
        saldo_final, trade_count = backtest(param_tuple, df)
        
        # Adicionar os resultados
        result = params.copy()
        result['saldo_final'] = saldo_final
        result['trade_count'] = trade_count
        results.append(result)
    
    # Converter os resultados para um DataFrame
    resultados_df = pd.DataFrame(results)
    return resultados_df

# Carregar o CSV com nome atualizado
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')
# df['time'] = pd.to_datetime(df['time'], unit='s')  # Descomente se necessário

# Definir a grade de parâmetros para a Random Search com intervalos e passos
param_grid_definicao = {
    'emaShortLength': [5, 20, 1],
    'emaLongLength': [20, 100, 1],
    'rsiLength': [7, 21, 1],
    'macdShort': [5, 15, 1],
    'macdLong': [20, 35, 1],
    'macdSignal': [5, 12, 1],
    'adxLength': [7, 28, 1],
    'adxSmoothing': [7, 21, 1],
    'adxThreshold': [20, 40, 1],
    'bbLength': [10, 30, 1],
    'bbMultiplier': [1.5, 3.0, 0.1],
    'lateralThreshold': [0.002, 0.01, 0.001],
    'stopgain_lateral_long': [1.02, 1.10, 0.01],
    'stoploss_lateral_long': [0.90, 0.98, 0.01],
    'stopgain_lateral_short': [0.90, 0.98, 0.01],
    'stoploss_lateral_short': [1.02, 1.10, 0.01],
    'stopgain_normal_long': [1.05, 1.35, 0.01],
    'stoploss_normal_long': [0.80, 0.95, 0.01],
    'stopgain_normal_short': [0.70, 0.95, 0.01],
    'stoploss_normal_short': [1.05, 1.15, 0.01],
    'commission': [0.00032]  # Fixa
}

# Gerar listas de valores para cada parâmetro
param_grid = {
    key: generate_parameter_list(*values) if key != 'commission' else values
    for key, values in param_grid_definicao.items()
}

# Definir o número de iterações para a Random Search
n_iterations = 10000  # Ajuste conforme a capacidade computacional

# Executar a Random Search
resultado_random = random_search(df, param_grid, n_iterations, seed=42)

# Ordenar os resultados pelo saldo final em ordem decrescente
resultado_random_sorted = resultado_random.sort_values(by='saldo_final', ascending=False)

# Exibir os top 5 resultados
print("\nTop 5 Resultados da Random Search:")
print(resultado_random_sorted.head(5))

# Salvar os resultados em um arquivo CSV para análise posterior
resultado_random_sorted.to_csv('random_search_resultados.csv', index=False)


Iniciando Random Search com 100 iterações...


Random Search: 100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


Top 5 Resultados da Random Search:
    emaShortLength  emaLongLength  rsiLength  macdShort  macdLong  macdSignal  \
70               7             85         17          6        26          10   
31              10             59          8         14        20           9   
19              12             54          9          6        32           5   
58               6             62         10          7        26           6   
57              19             50         20         10        23          10   

    adxLength  adxSmoothing  adxThreshold  bbLength  ...  \
70         18            18            40        14  ...   
31         25            17            32        22  ...   
19         22            10            26        24  ...   
58         24            10            38        16  ...   
57         24            21            40        21  ...   

    stoploss_lateral_long  stopgain_lateral_short  stoploss_lateral_short  \
70                    1.0              

In [9]:
import pandas as pd
import numpy as np
import talib
from numba import njit
from tqdm import tqdm
import time

# Definição da função MACD otimizada com Numba
@njit
def macd_calculation(series, fast_period, slow_period, signal_period):
    n = len(series)
    macd_line = np.empty(n)
    signal_line = np.empty(n)
    macd_hist = np.empty(n)
    ema_fast_arr = np.empty(n)
    ema_slow_arr = np.empty(n)
    
    # Inicialização das EMAs
    ema_fast = series[0]
    ema_slow = series[0]
    macd_line[0] = np.nan  # Diferença inicial como NaN
    signal_line[0] = np.nan  # Inicialização da linha de sinal como NaN
    macd_hist[0] = np.nan  # Inicialização do histograma como NaN
    ema_fast_arr[0] = ema_fast
    ema_slow_arr[0] = ema_slow
    
    multiplier_fast = 2.0 / (fast_period + 1)
    multiplier_slow = 2.0 / (slow_period + 1)
    multiplier_signal = 2.0 / (signal_period + 1)
    
    for i in range(1, n):
        # Cálculo da EMA rápida
        ema_fast = (series[i] - ema_fast) * multiplier_fast + ema_fast
        ema_fast_arr[i] = ema_fast
        # Cálculo da EMA lenta
        ema_slow = (series[i] - ema_slow) * multiplier_slow + ema_slow
        ema_slow_arr[i] = ema_slow
        # Cálculo da linha MACD
        macd_line[i] = ema_fast - ema_slow
        # Cálculo da linha de sinal
        if i < slow_period:
            signal_line[i] = np.nan  # Ainda não há dados suficientes para a linha de sinal
        elif i == slow_period:
            signal_line[i] = macd_line[i]  # Inicializa a linha de sinal com o primeiro valor da linha MACD
        else:
            signal_line[i] = (macd_line[i] - signal_line[i-1]) * multiplier_signal + signal_line[i-1]
        # Cálculo do histograma
        if i < slow_period:
            macd_hist[i] = np.nan  # Ainda não há dados suficientes para o histograma
        else:
            macd_hist[i] = macd_line[i] - signal_line[i]
    
    return ema_fast_arr, ema_slow_arr, macd_line, signal_line, macd_hist

def macd_optimizado_numba(series, fast_period=12, slow_period=26, signal_period=9):
    series_np = np.asarray(series, dtype=np.float64)  # Assegura que a série seja um array NumPy com float64
    return macd_calculation(series_np, fast_period, slow_period, signal_period)

# Definição da função MACD manual usando TA-Lib
def macd_func(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return ema_fast, ema_slow, macd_line, signal_line, macd_hist

def main():
    # Carregar o DataFrame
    try:
        df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')
    except FileNotFoundError:
        print("Erro: O arquivo 'BYBIT_BTCUSDT.P_1h.csv' não foi encontrado.")
        return
    except Exception as e:
        print(f"Erro ao carregar o arquivo CSV: {e}")
        return
    
    # Verificar se a coluna 'close' existe
    if 'close' not in df.columns:
        print("Erro: A coluna 'close' não foi encontrada no DataFrame.")
        return
    
    # Extrair os preços de fechamento
    series = df['close'].values
    
    # Verificar se a série contém dados suficientes
    if len(series) < 1:
        print("Erro: A série de preços de fechamento está vazia.")
        return
    
    # Número de iterações para benchmarking
    num_iterations = 100
    
    # Aquecer a função Numba (primeira chamada inclui tempo de compilação)
    print("Aquecendo a função Numba (primeira chamada pode ser mais lenta devido à compilação)...")
    macd_optimizado_numba(series)
    
    # Calcular MACD com TA-Lib para uma única execução e verificar resultados
    print("\nCalculando MACD usando TA-Lib (macd_func)...")
    ema_fast_manual, ema_slow_manual, macd_line_manual, signal_line_manual, macd_hist_manual = macd_func(series, 12, 26, 9)
    
    # Calcular MACD com Numba otimizado para uma única execução e verificar resultados
    print("Calculando MACD usando Numba otimizado (macd_optimizado_numba)...")
    ema_fast_numba, ema_slow_numba, macd_line_numba, signal_line_numba, macd_hist_numba = macd_optimizado_numba(series, 12, 26, 9)
    
    # Imprimir os primeiros 10 valores de cada linha para comparação
    print("\n📊 Primeiros 10 valores das linhas MACD, Sinal e Histograma:")
    print("🔹 MACD (TA-Lib):", macd_line_manual[:10])
    print("🔹 MACD (Numba):   ", macd_line_numba[:10])
    print("🔹 Sinal (TA-Lib):", signal_line_manual[:10])
    print("🔹 Sinal (Numba):  ", signal_line_numba[:10])
    print("🔹 Histograma (TA-Lib):", macd_hist_manual[:10])
    print("🔹 Histograma (Numba):  ", macd_hist_numba[:10])
    
    # Verificar se os resultados são similares, ignorando NaNs
    similarity_threshold = 1e-6
    comparison_mask = ~np.isnan(macd_line_manual) & ~np.isnan(macd_line_numba) & \
                      ~np.isnan(signal_line_manual) & ~np.isnan(signal_line_numba) & \
                      ~np.isnan(macd_hist_manual) & ~np.isnan(macd_hist_numba)
    
    if np.allclose(macd_line_manual[comparison_mask], macd_line_numba[comparison_mask], atol=similarity_threshold) and \
       np.allclose(signal_line_manual[comparison_mask], signal_line_numba[comparison_mask], atol=similarity_threshold) and \
       np.allclose(macd_hist_manual[comparison_mask], macd_hist_numba[comparison_mask], atol=similarity_threshold):
        print("✅ Ambas as implementações do MACD produzem resultados similares.")
    else:
        print("❌ As implementações do MACD produzem resultados diferentes.")
    
    # Benchmarking da função macd_func (TA-Lib)
    print("\n🔍 Iniciando o benchmark da função macd_func (TA-Lib)...")
    start_time = time.time()
    for _ in tqdm(range(num_iterations), desc='Benchmarking macd_func'):
        macd_func(series, 12, 26, 9)
    end_time = time.time()
    total_time_func = end_time - start_time
    iterations_per_sec_func = num_iterations / total_time_func
    print(f"🕒 macd_func: Tempo total para {num_iterations} iterações: {total_time_func:.4f} segundos")
    print(f"🚀 macd_func: Iterações por segundo: {iterations_per_sec_func:.2f}")
    
    # Benchmarking da função macd_optimizado_numba (Numba)
    print("\n🔍 Iniciando o benchmark da função macd_optimizado_numba (Numba otimizado)...")
    start_time = time.time()
    for _ in tqdm(range(num_iterations), desc='Benchmarking macd_optimizado_numba'):
        macd_optimizado_numba(series, 12, 26, 9)
    end_time = time.time()
    total_time_numba = end_time - start_time
    iterations_per_sec_numba = num_iterations / total_time_numba
    print(f"🕒 macd_optimizado_numba: Tempo total para {num_iterations} iterações: {total_time_numba:.4f} segundos")
    print(f"🚀 macd_optimizado_numba: Iterações por segundo: {iterations_per_sec_numba:.2f}")
    
    # Cálculo da velocidade de aceleração (Speedup)
    if total_time_numba > 0:
        speedup = total_time_func / total_time_numba
        print(f"\n⚡ Velocidade de aceleração (macd_func / macd_optimizado_numba): {speedup:.2f}x")
    else:
        print("\n⚡ Velocidade de aceleração não pode ser calculada (tempo de Numba otimizado é zero).")
    
    # Exibir métricas adicionais se necessário
    # Por exemplo, verificar a diferença média entre os resultados
    difference_macd = np.mean(np.abs(macd_line_manual[comparison_mask] - macd_line_numba[comparison_mask]))
    difference_signal = np.mean(np.abs(signal_line_manual[comparison_mask] - signal_line_numba[comparison_mask]))
    difference_hist = np.mean(np.abs(macd_hist_manual[comparison_mask] - macd_hist_numba[comparison_mask]))
    print("\n📊 Métricas adicionais de similaridade:")
    print(f"🔹 Diferença média na linha MACD: {difference_macd:.10f}")
    print(f"🔹 Diferença média na linha de sinal: {difference_signal:.10f}")
    print(f"🔹 Diferença média no histograma: {difference_hist:.10f}")

if __name__ == "__main__":
    main()


Aquecendo a função Numba (primeira chamada pode ser mais lenta devido à compilação)...

Calculando MACD usando TA-Lib (macd_func)...
Calculando MACD usando Numba otimizado (macd_optimizado_numba)...

📊 Primeiros 10 valores das linhas MACD, Sinal e Histograma:
🔹 MACD (TA-Lib): [nan nan nan nan nan nan nan nan nan nan]
🔹 MACD (Numba):    [        nan -6.38176638 -5.56540937 -4.74273184 -3.64527528 -0.74960106
  5.47634999  9.13512815 12.09701842 11.84668152]
🔹 Sinal (TA-Lib): [nan nan nan nan nan nan nan nan nan nan]
🔹 Sinal (Numba):   [nan nan nan nan nan nan nan nan nan nan]
🔹 Histograma (TA-Lib): [nan nan nan nan nan nan nan nan nan nan]
🔹 Histograma (Numba):   [nan nan nan nan nan nan nan nan nan nan]
❌ As implementações do MACD produzem resultados diferentes.

🔍 Iniciando o benchmark da função macd_func (TA-Lib)...


Benchmarking macd_func: 100%|██████████| 100/100 [00:00<00:00, 3824.83it/s]


🕒 macd_func: Tempo total para 100 iterações: 0.0282 segundos
🚀 macd_func: Iterações por segundo: 3551.00

🔍 Iniciando o benchmark da função macd_optimizado_numba (Numba otimizado)...


Benchmarking macd_optimizado_numba: 100%|██████████| 100/100 [00:00<00:00, 6235.96it/s]

🕒 macd_optimizado_numba: Tempo total para 100 iterações: 0.0180 segundos
🚀 macd_optimizado_numba: Iterações por segundo: 5543.99

⚡ Velocidade de aceleração (macd_func / macd_optimizado_numba): 1.56x

📊 Métricas adicionais de similaridade:
🔹 Diferença média na linha MACD: 0.0004701391
🔹 Diferença média na linha de sinal: 0.0007324493
🔹 Diferença média no histograma: 0.0002623102
